# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import nltk
import re
nltk.download(['punkt','stopwords','wordnet'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
def load_data(database_filepath='yfdb.db',tablename='table_df'):
    """
    INPUT:
        database_filepath (string) : database location
    OUTPUT:
        X  : messages to process
        Y  : training/evaluating categories
        labels  : list of message classification labels
    """
    engine = create_engine('sqlite:///' + database_filepath)
    df = pd.read_sql_table(tablename, con = engine)
    X = df['message'] 
    Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)
    return X, Y,list(Y.columns)

In [3]:
X,Y,category_names=load_data()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    Remove capitalization and special characters and lemmatize texts
    Args:
    text: string. String containing message for processing
    
    Returns:
    clean_tokens: list of strings. List containing normalized and stemmed word tokens
    """ 
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
   pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10,
                                                             min_samples_split = 10)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Split data into train and test tests.
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,  random_state = 42)

# Train the model.
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Test the model and print the classification report for each of the 36 categories.
def performance(model, X_test, y_test,category_names):
    y_pred = model.predict(X_test)
    print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=category_names))
   
performance(pipeline, X_test, y_test,category_names)

                        precision    recall  f1-score   support

               related       0.85      0.45      0.59       895
               request       0.00      0.00      0.00        26
                 offer       0.72      0.62      0.67      2131
           aid_related       0.57      0.09      0.16       422
          medical_help       0.67      0.07      0.13       270
      medical_products       0.75      0.02      0.05       127
     search_and_rescue       1.00      0.01      0.02        88
              security       0.65      0.10      0.17       155
              military       0.00      0.00      0.00         0
           child_alone       0.83      0.31      0.45       339
                 water       0.84      0.43      0.57       595
                  food       0.78      0.27      0.40       470
               shelter       0.33      0.01      0.03        73
              clothing       0.80      0.08      0.14       104
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {#'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 50], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1,verbose=2)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, total=  47.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   57.0s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, total=  47.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, total=  47.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, total=  46.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, total=  46.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, total=  47.5s
[CV] clf__estimator__min_samples_split=2, clf__estimato

[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 72.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 50], 'clf__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [ ]:
cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
performance(cv, X_test, y_test,category_names)

                        precision    recall  f1-score   support

               related       0.89      0.40      0.55       895
               request       0.00      0.00      0.00        26
                 offer       0.78      0.60      0.68      2131
           aid_related       0.61      0.05      0.10       422
          medical_help       0.74      0.05      0.10       270
      medical_products       0.67      0.03      0.06       127
     search_and_rescue       0.25      0.01      0.02        88
              security       0.62      0.03      0.06       155
              military       0.00      0.00      0.00         0
           child_alone       0.88      0.22      0.35       339
                 water       0.88      0.42      0.57       595
                  food       0.83      0.20      0.32       470
               shelter       0.86      0.08      0.15        73
              clothing       0.89      0.08      0.14       104
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

# Train the adjusted pipeline. 
pipeline2.fit(X_train, y_train)

# Check the performance of the adjusted model.
performance(pipeline2, X_test, y_test,category_names)

                        precision    recall  f1-score   support

               related       0.79      0.52      0.63       895
               request       0.00      0.00      0.00        26
                 offer       0.75      0.59      0.66      2131
           aid_related       0.57      0.25      0.35       422
          medical_help       0.66      0.30      0.41       270
      medical_products       0.55      0.18      0.27       127
     search_and_rescue       0.06      0.01      0.02        88
              security       0.52      0.31      0.39       155
              military       0.00      0.00      0.00         0
           child_alone       0.75      0.61      0.67       339
                 water       0.83      0.68      0.75       595
                  food       0.77      0.55      0.64       470
               shelter       0.69      0.33      0.44        73
              clothing       0.48      0.30      0.37       104
                 money       0.79      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [32]:
parameters2 = { 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

cv2 = GridSearchCV(pipeline2, param_grid=parameters2)

cv2.fit(X_train, y_train)
performance(cv2, X_test, y_test,category_names)


                        precision    recall  f1-score   support

               related       0.76      0.54      0.63       895
               request       0.25      0.04      0.07        26
                 offer       0.76      0.62      0.68      2131
           aid_related       0.56      0.27      0.37       422
          medical_help       0.60      0.31      0.41       270
      medical_products       0.42      0.15      0.22       127
     search_and_rescue       0.06      0.01      0.02        88
              security       0.47      0.30      0.36       155
              military       0.00      0.00      0.00         0
           child_alone       0.74      0.62      0.68       339
                 water       0.83      0.65      0.73       595
                  food       0.79      0.57      0.66       470
               shelter       0.60      0.37      0.46        73
              clothing       0.44      0.33      0.37       104
                 money       0.63      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [12]:
def save_model(model, model_filepath):
    """
    Saves the model to a Python pickle file    
    Args:
    model: Trained model
    model_filepath: Filepath to save the model
    """

    # save model to pickle file
    pickle.dump(model, open(model_filepath, 'wb'))

    
save_model(cv,'classifer.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [33]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import nltk
import re
nltk.download(['punkt','stopwords','wordnet'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier


# load data from database
def load_data(database_filepath='yfdb.db',tablename='table_df'):
    """
    INPUT:
        database_filepath (string) : database location
    OUTPUT:
        X  : messages to process
        Y  : training/evaluating categories
        labels  : list of message classification labels
    """
    engine = create_engine('sqlite:///' + database_filepath)
    df = pd.read_sql_table(tablename, con = engine)
    X = df['message'] 
    Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)
    return X, Y,list(Y.columns)

def tokenize(text):
    """
    Remove capitalization and special characters and lemmatize texts
    Args:
    text: string. String containing message for processing
    
    Returns:
    clean_tokens: list of strings. List containing normalized and stemmed word tokens
    """ 
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

def build_model(): 
    """
    Function: build model that consist of pipeline
    Args:
      N/A
    Return
      cv(model): Grid Search model 
    """
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10,
                                                             min_samples_split = 10)))
    ])
    
    parameters = {#'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 50], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

    cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1,verbose=2)

   # cv.fit(X_train, y_train)
    return cv

def performance(model, X_test, y_test,category_names):
    y_pred = model.predict(X_test)
    print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=category_names))

def save_model(cv):
    """
    Function: save model as pickle file.
    Args:
      cv:target model
    Return:
      N/A
    """
    with open('classifer.pkl', 'wb') as f:
        pickle.dump(cv, f)   
        
def main(): 
    print("Load data")
    X,Y,category_names=load_data()
    X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2, random_state=42)
    print("Build model")
    model=build_model()
    print("Train model")
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Evaluation model")
    performance(model, X_test, y_test,category_names)
    print("Save model")
    save_model(model)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
if __name__ == "__main__":
    main()